![MLU Logo](../../data/MLU_Logo.png)

# <a name="0">Machine Learning Accelerator - Computer Vision - Lecture 2</a>


## Customized Image Classification with AutoGluon

In this tutorial, we load images and the corresponding labels into [AutoGluon](https://autogluon.mxnet.io/index.html) and use this data to obtain a neural network that can classify new images. This is different from traditional machine learning where we need to manually define the neural network and then specify the hyperparameters in the training process. Instead, with just a single call to AutoGluon’s fit function, AutoGluon automatically trains many models with different hyperparameter configurations and returns the model that achieved the highest level of accuracy.

Note: Please use **GPU** for training. CPU training will lead to an unceasing running script. 

In [ ]:
! pip install -q -r ../../requirements.txt

Let's import the ImagePredictor

In [ ]:
from autogluon.multimodal import MultiModalPredictor
from autogluon.core.utils.loaders import load_zip
import os
import pandas as pd

To use AutoGluon for computer vision task training, we need to organize our data with the following structure:

    data/
    ├── train/
        ├── class1/
        ├── class2/
        ├── class3/
        ├── ...
    ├── test/
        ├── class1/
        ├── class2/
        ├── class3/
        ├── ...

Here each subfolder contains all images that belong to that category, e.g., `class1` contains all images belonging to the first class. We generally recommend at least 100 training images per class for reasonable classification performance, but this might depend on the type of images in your specific use-case.

## 1. <a name="1">Download the dataset</a>
(<a href="#0">Go to top</a>)

For demonstration purposes, we use a subset of the [Shopee-IET](https://www.kaggle.com/c/shopee-iet-machine-learning-competition/data) dataset from Kaggle. Each image in this data depicts a clothing item and the corresponding label specifies its clothing category. Our subset of the data contains the following possible labels: BabyPants, BabyShirt, womencasualshoes, womenchiffontop.

In [ ]:
download_dir = './ag_image_dataset'
zip_file = 'https://autogluon.s3.amazonaws.com/datasets/shopee-iet.zip'

load_zip.unzip(zip_file, unzip_dir=download_dir)

If you use this on your own dataset, just point it to your training or test folder.

In [ ]:
# Create a dataframe from dataset
training_folder = os.path.join(download_dir, 'data/train')
test_folder = os.path.join(download_dir, 'data/test')


def process_dataset(data_dir):
    data = {}
    for category in os.listdir(data_dir):
        category_dir = os.path.join(data_dir, category)
        data[category] =  [os.path.join(category_dir, path) for path in os.listdir(category_dir)]
    
    # Create a dataframe from dict
    # Create an empty list to store the rows
    rows = []

    # Iterate through the dictionary
    for key, values in data.items():
        # For each value in the list, create a row with the value and the key
        for value in values:
            rows.append({'Path': value, 'label': key})

    # Create the DataFrame from the list of rows
    df = pd.DataFrame(rows)
    return df

train_data = process_dataset(training_folder)
test_data = process_dataset(test_folder)

In [ ]:
train_data.head()

Let's print the training dataset

## 2. <a name="2">Use AutoGluon to Fit Models</a>
(<a href="#0">Go to top</a>)

Now, let's fit a __classifier__ using AutoGluon [predictor.fit()](https://auto.gluon.ai/stable/tutorials/image_prediction/beginner.html). Within fit, the dataset is __automatically__ split into training and validation sets. The model with the best hyperparameter configuration is selected based on its performance on the __validation set__.

In [ ]:
predictor = MultiModalPredictor(label='label')

predictor.fit(
    train_data=train_data,
    time_limit=10 * 60 # how long fit() should run (in seconds)
)

## 3. <a name="3">Model Results</a>
(<a href="#0">Go to top</a>)

Use AutoGluon to Fit Models
Autogluon also provides the training results, which can be accessed by calling `predictor.fit_summary()`. 

In [ ]:
fit_result = predictor.fit_summary()

In [ ]:
fit_result

We can access certain results from this summary. For example, training and validation accuracies below.

In [ ]:
print('Train time: %.3f, val acc: %.3f' %(fit_result['training_time'], fit_result['val_accuracy']))

 ## 4. <a name="4">Making Predictions</a>
(<a href="#0">Go to top</a>)

We can call the predict function to run on different images.

Let's get predictions on the test set.

In [ ]:
pred = predictor.predict(test_data)
print(pred)